In [9]:
import pandas as pd

# Чтение локального файла
df = pd.read_csv("processed_asthma.csv")

# Просмотр первых 5 строк
df.head()


,Patient_ID,Age,Gender,BMI,Smoking_Status,Family_History,Allergies,Air_Pollution_Level,Physical_Activity_Level,Occupation_Type,Comorbidities,Medication_Adherence,Number_of_ER_Visits,Peak_Expiratory_Flow,FeNO_Level,Has_Asthma,Asthma_Control_Level
0,ASTH100000,52,Female,27.6,Former,1,NaN,Moderate,Sedentary,Outdoor,Diabetes,0.38,0,421.0,46.0,0,NaN
1,ASTH100001,15,Male,24.6,Former,0,Dust,Low,Moderate,Indoor,Both,0.60,2,297.6,22.9,0,NaN
2,ASTH100002,72,Female,17.6,Never,0,NaN,Moderate,Moderate,Indoor,NaN,0.38,0,303.3,15.3,0,NaN
3,ASTH100003,61,Male,16.8,Never,0,Multiple,High,Sedentary,Outdoor,Both,0.60,1,438.0,40.1,1,Poorly Controlled
4,ASTH100004,21,Male,30.2,Never,0,NaN,Moderate,Active,Indoor,NaN,0.82,3,535.0,27.7,0,NaN


In [2]:
# Проверка структуры данных
df.info()

# Основная статистика для числовых столбцов
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Patient_ID               10000 non-null  object 
 1   Age                      10000 non-null  int64  
 2   Gender                   10000 non-null  object 
 3   BMI                      10000 non-null  float64
 4   Smoking_Status           10000 non-null  object 
 5   Family_History           10000 non-null  int64  
 6   Allergies                7064 non-null   object 
 7   Air_Pollution_Level      10000 non-null  object 
 8   Physical_Activity_Level  10000 non-null  object 
 9   Occupation_Type          10000 non-null  object 
 10  Comorbidities            5033 non-null   object 
 11  Medication_Adherence     10000 non-null  float64
 12  Number_of_ER_Visits      10000 non-null  int64  
 13  Peak_Expiratory_Flow     10000 non-null  float64
 14  FeNO_Level             

,Age,BMI,Family_History,Medication_Adherence,Number_of_ER_Visits,Peak_Expiratory_Flow,FeNO_Level,Has_Asthma
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,44.930700,25.053320,0.303400,0.497998,1.015900,400.884090,25.101420,0.243300
std,25.653559,4.874466,0.459749,0.224809,1.020564,97.531113,9.840184,0.429096
min,1.000000,15.000000,0.000000,0.000000,0.000000,150.000000,5.000000,0.000000
25%,23.000000,21.600000,0.000000,0.320000,0.000000,334.800000,18.200000,0.000000
50%,45.000000,25.000000,0.000000,0.500000,1.000000,402.500000,25.000000,0.000000
75%,67.000000,28.400000,1.000000,0.670000,2.000000,468.700000,31.700000,0.000000
max,89.000000,45.000000,1.000000,0.990000,6.000000,600.000000,63.900000,1.000000


In [3]:
# Подсчет пропущенных значений
missing = df.isnull().sum()

# Процент пропусков
missing_percent = (missing / len(df)) * 100

# Создаем таблицу с результатами
missing_df = pd.DataFrame({
    'Пропущенные значения': missing,
    'Процент %': missing_percent
})

missing_df


,Пропущенные значения,Процент %
Patient_ID,0,0.00
Age,0,0.00
Gender,0,0.00
BMI,0,0.00
Smoking_Status,0,0.00
Family_History,0,0.00
Allergies,2936,29.36
Air_Pollution_Level,0,0.00
Physical_Activity_Level,0,0.00
Occupation_Type,0,0.00


In [4]:
# Выбираем числовые столбцы
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Расчет IQR для выявления выбросов
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Подсчет выбросов для каждого числового столбца
outliers = ((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))).sum()
outliers_percent = (outliers / len(df)) * 100

outliers_df = pd.DataFrame({
    'Выбросы': outliers,
    'Процент %': outliers_percent
})
outliers_df


,Выбросы,Процент %
Age,0,0.00
BMI,24,0.24
Family_History,0,0.00
Medication_Adherence,0,0.00
Number_of_ER_Visits,3,0.03
Peak_Expiratory_Flow,0,0.00
FeNO_Level,41,0.41
Has_Asthma,2433,24.33


In [10]:
# Создаем сводную таблицу с пропусками и выбросами
metrics_df = pd.DataFrame({
    'Пропущенные значения %': (df.isnull().sum() / len(df)) * 100,
    'Выбросы %': ((df[numeric_cols] < (Q1 - 1.5*IQR)) | (df[numeric_cols] > (Q3 + 1.5*IQR))).sum() / len(df) * 100
})

# Для категориальных столбцов добавляем только пропуски
for col in df.select_dtypes(include=['object']).columns:
    if col not in metrics_df.index:
        metrics_df.loc[col] = [df[col].isnull().sum() / len(df) * 100, 0]

metrics_df = metrics_df.sort_index()
metrics_df



,Пропущенные значения %,Выбросы %
Age,0.00,0.00
Air_Pollution_Level,0.00,NaN
Allergies,29.36,NaN
Asthma_Control_Level,75.67,NaN
BMI,0.00,0.24
Comorbidities,49.67,NaN
Family_History,0.00,0.00
FeNO_Level,0.00,0.41
Gender,0.00,NaN
Has_Asthma,0.00,24.33
